In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 10, kernel_size=3) #input - 28x28x1, output - 26x26x10, RF - 3x3
        self.bn1 = nn.BatchNorm2d(10)
        self.conv2 = nn.Conv2d(10, 10, kernel_size=3) #input - 26x26x10, output - 24x24x10, RF - 5x5
        self.bn2 = nn.BatchNorm2d(10)
        self.pool1 = nn.MaxPool2d(2, 2) #input - 24x24x10, output - 12x12x10, RF - 10x10
        self.dropout1 = nn.Dropout(0.25)

        self.conv3 = nn.Conv2d(10, 20, kernel_size=3) #input - 12x12x10, output - 10x10x20, RF - 12x12
        self.bn3 = nn.BatchNorm2d(20)
        self.conv4 = nn.Conv2d(20, 20, kernel_size=3) #input - 10x10x20, output - 8x8x20, RF - 14x14
        self.bn4 = nn.BatchNorm2d(20)
        self.pool2 = nn.MaxPool2d(2, 2) #input - 8x8x20, output - 4x4x20, RF - 28x28
        self.dropout2 = nn.Dropout(0.25)

        self.fc1 = nn.Linear(4*4*20, 10) #input - 4*4*20, output - 10

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(self.bn2(F.relu(self.conv1(x))))))
        x = self.dropout1(x)
        x = self.pool2(F.relu(self.conv4(self.bn4(F.relu(self.conv3(x))))))
        x = self.dropout2(x)
        x = x.view(-1, 4*4*20)
        x = self.fc1(x)
        return F.log_softmax(x, dim=1)


In [ ]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]             100
       BatchNorm2d-2           [-1, 10, 26, 26]              20
            Conv2d-3           [-1, 10, 24, 24]             910
         MaxPool2d-4           [-1, 10, 12, 12]               0
           Dropout-5           [-1, 10, 12, 12]               0
            Conv2d-6           [-1, 20, 10, 10]           1,820
       BatchNorm2d-7           [-1, 20, 10, 10]              40
            Conv2d-8             [-1, 20, 8, 8]           3,620
         MaxPool2d-9             [-1, 20, 4, 4]               0
          Dropout-10             [-1, 20, 4, 4]               0
           Linear-11                   [-1, 10]           3,210
Total params: 9,720
Trainable params: 9,720
Non-trainable params: 0


In [ ]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


100%|██████████| 9912422/9912422 [00:00<00:00, 207689558.72it/s]


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 37643161.54it/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 62219006.03it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 4314955.55it/s]


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.1000354215502739 batch_id=468: 100%|██████████| 469/469 [00:17<00:00, 27.26it/s]



Test set: Average loss: 0.0580, Accuracy: 9810/10000 (98%)



loss=0.0869290828704834 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 29.36it/s]



Test set: Average loss: 0.0484, Accuracy: 9842/10000 (98%)



loss=0.07624214142560959 batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 28.18it/s]


...


Test set: Average loss: 0.0200, Accuracy: 9935/10000 (99%)



loss=0.021479086950421333 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 30.13it/s]



Test set: Average loss: 0.0216, Accuracy: 9918/10000 (99%)



loss=0.038445696234703064 batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 30.61it/s]



Test set: Average loss: 0.0211, Accuracy: 9935/10000 (99%)

